### Define the Prompt

We define a prompt that can perform two tasks given a topic and an argument <br>
1) classify whether the argument is a fallacy, and if so what fallacy the  argument contains <br>
2) decide whether the argument supports or refutes the topic <br>
Prompts templates are defined by a string where ```{}``` is a placeholder for where the inputs for a sample are to be and ```<mask>``` is filled in as the predicted label. <br>
*Multiple masked tokens in a single prompt is currently not supported*

In [4]:
templates = {"fallacy": "fallacy task. Topic: {} Text: {} This contains the fallacy: <mask>", 
             "stance": "procon task. Topic: {} Text: {} Has the relation: <mask>"}

def fallacy_policy(pred):
  fallacies = {'AppealtoEmotion', 'RedHerring', 'NoFallacy', 'IrrelevantAuthority','AdHominem','HastyGeneralization'}
  if pred not in fallacies: return 'UNKNOWN'
  return pred
def stance_policy(pred):
  if pred not in {"support", "contradict"}: return "UNKNOWN"
  return pred

policies = {"fallacy": fallacy_policy, "stance": stance_policy}

argument_prompt = Prompt(templates, policies)

RobertaPrompt uses this class to convert a sample into it's desire prompt - for example

In [4]:
argument_prompt.test_sample(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "fallacy")

'fallacy task. Topic: Should we allow animal testing? Text: Animal testing abuses animals and should be dis-continued This contains the fallacy: <mask>'

In [5]:
argument_prompt.train_sample(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "NoFallacy", "fallacy")

'fallacy task. Topic: Should we allow animal testing? Text: Animal testing abuses animals and should be dis-continued This contains the fallacy: NoFallacy'

In [6]:
print(argument_prompt)

=== Prompts ===
Task: fallacy, Template: fallacy task. Topic: {} Text: {} This contains the fallacy: <mask>
Task: stance, Template: procon task. Topic: {} Text: {} Has the relation: <mask>


### Load Model
We load a model we have already trained for this task. Some sample predictions are also displayed

In [ ]:
pmodel = RobertaPrompt(model='/content/drive/MyDrive/Laidlaw Research Project/models/prompt_combined', device = torch.device('cuda'), prompt = argument_prompt)

In [ ]:
print(pmodel)

======== Base Model ============
/content/drive/MyDrive/Laidlaw Research Project/models/prompt_combined

======== Tasks ============
Task: fallacy, Template: fallacy task. Topic: {} Text: {} This contains the fallacy: <mask>
Task: stance, Template: procon task. Topic: {} Text: {} Has the relation: <mask>



In [ ]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: NoFallacy
Stance: contradict


In [ ]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Your stupid for bringing this up, animal testing is horrible"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Your stupid for bringing this up, animal testing is horrible"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: AdHominem
Stance: contradict


In [ ]:
fallacy = pmodel.infer(["Should we allow animal testing?", "My Dad had a dog once, he says animal testing should be allowed"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "My Dad had a dog once, he says animal testing should be allowed"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: IrrelevantAuthority
Stance: support


In [ ]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Everyone has a favorite animal, what is yours?"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Everyone has a favorite animal, what is yours?"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: RedHerring
Stance: support


In [ ]:
print(pmodel.test("/content/drive/MyDrive/Laidlaw Research Project/data/test_samples.tsv"))

macro f1: 0.7863247863247864
micro f1: 0.7933884297520661
weighted f1: 0.7943516538557861
              precision    recall  f1-score   support

  contradict       0.73      0.77      0.75        48
     support       0.84      0.81      0.83        73

    accuracy                           0.79       121
   macro avg       0.78      0.79      0.79       121
weighted avg       0.80      0.79      0.79       121



## Train Model

In [20]:
pmodel = RobertaPrompt(model='roberta-large', device = torch.device('cuda'), prompt = argument_prompt)
print(pmodel)

======== Base Model ============
roberta-large

======== Tasks ============
Task: fallacy, Template: fallacy task. Topic: {} Text: {} This contains the fallacy: <mask>
Task: stance, Template: procon task. Topic: {} Text: {} Has the relation: <mask>



In [21]:
pmodel.train("/content/drive/MyDrive/Laidlaw Research Project/data/stance/training.tsv", "/content/drive/MyDrive/Laidlaw Research Project/data/stance/val.tsv", output_dir="/content/drive/MyDrive/Laidlaw Research Project/data/stance/model", epochs=3)


======== Epoch 1 / 3 ========
Training...
  Batch    10  of     62.    Elapsed: 0:00:11.
  Batch    20  of     62.    Elapsed: 0:00:19.
  Batch    30  of     62.    Elapsed: 0:00:27.
  Batch    40  of     62.    Elapsed: 0:00:35.
  Batch    50  of     62.    Elapsed: 0:00:43.
  Batch    60  of     62.    Elapsed: 0:00:52.

  Average training loss: 2.67
  Training epcoh took: 0:00:53

Running Validation...
SAVING NEW MODEL ... 
  Validation Loss: 0.01
  Validation took: 0:00:31

======== Epoch 2 / 3 ========
Training...
  Batch    10  of     62.    Elapsed: 0:00:08.
  Batch    20  of     62.    Elapsed: 0:00:16.
  Batch    30  of     62.    Elapsed: 0:00:25.
  Batch    40  of     62.    Elapsed: 0:00:33.
  Batch    50  of     62.    Elapsed: 0:00:41.
  Batch    60  of     62.    Elapsed: 0:00:49.

  Average training loss: 0.01
  Training epcoh took: 0:00:50

Running Validation...
SAVING NEW MODEL ... 
  Validation Loss: 0.00
  Validation took: 0:00:08

======== Epoch 3 / 3 ========
Tra

{1: {'Training Loss': 2.665111167721578,
  'Valid. Loss': 0.0055667326669208705,
  'Training Time': '0:00:53',
  'Validation Time': '0:00:31'},
 2: {'Training Loss': 0.00734625298965482,
  'Valid. Loss': 0.00396701754652895,
  'Training Time': '0:00:50',
  'Validation Time': '0:00:08'},
 3: {'Training Loss': 0.004069779668146024,
  'Valid. Loss': 0.003757576982025057,
  'Training Time': '0:00:50',
  'Validation Time': '0:00:09'}}

In [22]:
print(pmodel.test("/content/drive/MyDrive/Laidlaw Research Project/data/stance/test_samples.tsv"))

macro f1: 0.7993366500829187
micro f1: 0.8016528925619834
weighted f1: 0.8037909625426586
              precision    recall  f1-score   support

  contradict       0.70      0.88      0.78        48
     support       0.90      0.75      0.82        73

    accuracy                           0.80       121
   macro avg       0.80      0.81      0.80       121
weighted avg       0.82      0.80      0.80       121

